In [6]:
!wget https://raw.githubusercontent.com/zaidalyafeai/QuickDraw10/master/quick_draw10.py

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import os
import glob
import numpy as np
import torch
from quick_draw10 import download_and_load, plot_tsne
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, TensorDataset,DataLoader

In [3]:
x_train, y_train, x_test, y_test, class_names = download_and_load()

downloading ...
loading ...
Training Data :  80000
Testing  Data :  20000


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [5]:
image_size = 28

# Reshape and normalize
x_train = x_train.reshape(x_train.shape[0], image_size, image_size, 1).astype('float32')
x_test = x_test.reshape(x_test.shape[0], image_size, image_size, 1).astype('float32')

x_train /= 255.0
x_test /= 255.0



In [18]:
batch_size = 128

In [32]:
testloader = DataLoader(TensorDataset(torch.Tensor(x_test),torch.LongTensor(y_test)),batch_size =32)

In [20]:
trainloader = DataLoader(TensorDataset(torch.Tensor(x_train),torch.LongTensor(y_train)),batch_size =batch_size)

In [21]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,16,kernel_size = 3)
        self.pool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(16,32,kernel_size = 3)
        self.pool2 = nn.MaxPool2d(2)
        self.conv3 = nn.Conv2d(32,64,3)
        self.pool3 = nn.MaxPool2d(2)
        self.fc1 = nn.Linear(64,len(class_names))
        self.fc2 = nn.Linear(len(class_names),len(class_names))
        self.act = nn.ReLU()
    
    def forward(self,x):
#         print("init "+str(x.shape))
        x = self.act(self.conv1(x))
#         print("Conv1 "+str(x.shape))
        x = self.pool1(x)
        x = self.act(self.conv2(x))
#         print("Conv2 "+str(x.shape))
        x = self.pool2(x)
        x = self.act(self.conv3(x))
#         print("Conv3 "+str(x.shape))
        x = self.pool3(x)
#         print("Pool3 "+str(x.shape))
        x = x.view(-1,64)
#         print("Flat "+str(x.shape))
        x = torch.tanh(self.fc1(x))
#         print("Tanh "+str(x.shape))
        x = F.softmax(self.fc2(x),dim=0)
#         print("softmax "+str(x.shape))
        return x
        

In [22]:
net = Net()
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())

In [34]:
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader,0):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
#         print(inputs.shape)
        inputs = torch.reshape(inputs,[batch_size, 1,28, 28])
#         print(inputs.shape)
        optimizer.zero_grad()
        outputs = net(inputs)
#         print(outputs.shape)
        loss = criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        

In [33]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
#         print(images.shape)
        images = torch.reshape(images,[32, 1,28, 28])
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 20000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 20000 test images: 82 %
